# Vergleich von Imputation Methoden

An dieser Stelle sollen verschiede Methoden zum interpolieren von fehlenden Werten betrachtet und verglichen werden.

### Vorbereitung

In [7]:
import pandas as pd
import numpy as np
import math
import time

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

In der Folge werden keine Regionen oder Gruppierungen von Ländern und nur die Jahre ab 1990 betrachtet. 
Von dem verbliebenen Datensatz werden nur Indikatoren behalten, die mindesten 20% gefüllt sind und nach ihrer Relevanz manuell ausgewählt wurden (siehe 03_preparing_data).

In [8]:
def reset_base():
    base= pd.read_csv('additional_data/base.csv') 
    base.set_index(['Country Name', 'Indicator Name'], inplace=True)
    base = base.sort_index(level=['Country Name', 'Indicator Name'])
    return base

In [9]:
base = reset_base()
base.isna().sum().sum()

170307

Um die Performanz unterschiedlicher Imutation Verfahren zu vergleichen werden weitere 1000 vorhandene (nicht NaN) Einträge entfernt. Diese 1000 Einträge werden später als Test-Daten verwendet, auf ihrer Grundlage lassen sich die Fehler der analysierten Verfahren errechnen. Um sicherzustellen, dass die Ergebnisse reproduzierbar sind wird ein Random State gesetzt. Dann werden zufällig Koordinaten zu Dateneinträgen gezogen. Da an dieser Stelle nur vorhandene Einträge relevant sind, werde zunächst zu viele Koordinaten gezogen, diese dann mit dem Datensatz abgeglichen und gelöscht, falls sie zu einem NaN zeigen und dann die ersten 1000 verbliebenen (und damit relevanten) Einträge ausgewählt. Diese werden in den Trainingsdaten entfernt. Auf diese Weise bleibt eine Reproduzierbarkeit erhalten.

In [10]:
base.head(3)

1990  1991  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1992  1993  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1994  1995  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1996  1997  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 1998  1999  \
Country Name Indicator Name                                                   
Afghanistan  Access to clean fuels and technologies for cook...   NaN   NaN   
             Access to electricity (% of population)              NaN   NaN   
             Access to electricity, rural (% of rural popula...   NaN   NaN   

                                                                 ...  \
Country Name Indicator Name                                      ...   
Afghanistan  Access to clean fuels and technologies for cook...  ...   
             Access to electricity (% of population)             ...   
             Access to electricity, rural (% of rural popula...  ...   

                                                                      2011  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  21.500000   
             Access to electricity (% of population)             43.222019   
             Access to electricity, rural (% of rural popula...  29.572881   

                                                                      2012  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  23.000000   
             Access to electricity (% of population)             69.100000   
             Access to electricity, rural (% of rural popula...  60.849157   

                                                                      2013  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  24.799999   
             Access to electricity (% of population)             68.982941   
             Access to electricity, rural (% of rural popula...  61.315788   

                                                                      2014  \
Country Name Indicator Name                                                  
Afghanistan  Access to clean fuels and technologies for cook...  26.700001   
             Access to electricity (% of population)             89.500000   
             Access to electricity, rural (% of rural popula...  86.500512   

                                                                      2015  \
Country Name Indicator Name                                                  
Afgh

### Simulation fehlender Werte und Evaluation

In [11]:
def get_cords(frac):
    n = int(base.isna().sum().sum()*frac)
    print(f'Testdaten mit {frac*100}% fehlenden Werten (absolut: {n})')
    #random state to ensure reproducibility
    rnds = np.random.RandomState(n)

    #coordinates for data entries to be removed randomly
    #5000 entries are selected
    cords = pd.DataFrame([[rnds.randint(0, len(base), size=n*4)[i], 
                  rnds.randint(0, len(base.columns), size=n*4)[i]]
                  for i in range(n*4)])

    #all coordinates pointing to NaN entries are removed and
    #first 1000 remaining entries are selected
    cords['value'] = [base.iloc[cords[0][i], cords[1][i]] for i in cords.index]
    cords = cords.dropna()[:n].reset_index(drop=True)
    
    return cords

In [12]:
cords = get_cords(0.1)

Testdaten mit 10.0% fehlenden Werten (absolut: 17030)


In [13]:
#getting train data by changing randomly chosen values to NaN
def reset_train():
    train = base.copy()
    for i in cords.index:
        train.iloc[cords[0][i], cords[1][i]] = None
    return train

In [14]:
results = []

In [15]:
def evaluate(method, df, t):
    
    '''
    res1 = (pd.DataFrame({'y_true': [base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                        'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                        'indicator': [df.index.get_level_values('Indicator Name')[cords[0][i]] for i in cords.index],
                        'year': [df.columns[cords[1][i]] for i in cords.index]})
         )
    '''
    
    #scaling original data and imputed data
    #necessary ?????????????????????????????????????
    train = reset_train()
    scaler = StandardScaler().fit(train) #fitting on train?
    norm_base = pd.DataFrame(scaler.transform(base))
    df = pd.DataFrame(scaler.transform(df))

    #getting imputed values for simulated NaNs and true value 
    res =pd.DataFrame({'y_true': [norm_base.iloc[cords[0][i], cords[1][i]] for i in cords.index],
                       'y_pred': [df.iloc[cords[0][i], cords[1][i]] for i in cords.index]
                      })
    res = res.dropna()

   
    #calculate evaluation metrics
    r2 = r2_score(res['y_true'], res['y_pred'])
    rmse = math.sqrt(mean_squared_error(res['y_true'], res['y_pred']))
    still_missing = df.isna().sum().sum()
    
    print(f'Mit dieser Methode bleiben {still_missing} NaNs bestehen.')
    print('')
    print(f'{len(res)} Werte wurden für die Metriken verwendet.')
    print(f'r2: {r2}, rmse: {rmse}')
    
    results.append([method, r2, rmse, still_missing, t])


###  Imputation Verfahren

Es werden diese Imputation Methoden verglichen:
- Backcasting
- Durchschnitt
- regionaler Durchschnitt
- Iterative Imputer
- KNN Imputer

#### Backfill

In [16]:
def impute_backfill(df):
    df = df.fillna(method='bfill', limit=3)
    return df

In [17]:
base = reset_base()
train = reset_train()



t0 = time.time()
df= impute_backfill(train) 
t1 = time.time()

t = t1-t0


evaluate('Backfill', df, t)

Mit dieser Methode bleiben 33605 NaNs bestehen.

16462 Werte wurden für die Metriken verwendet.
r2: -0.043800632990352195, rmse: 2.089040474302827


#### Durchschnitt des Indikators über alle Jahre hinweg

In [18]:
def impute_overall_means(df):
    #fill NaNs with overall mean of that indicator
    values = pd.DataFrame(df.stack()).groupby('Indicator Name')[0].mean()
    df = pd.DataFrame(df.stack(dropna=False))
    
    df[0] = df[0].fillna(df.groupby('Indicator Name')[0].transform('mean'))
    df = df.unstack()
    df.columns = df.columns.droplevel(0)
    df = df.sort_index(level='Indicator Name')
        
    return df

In [19]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = impute_overall_means(train)
t1 = time.time()

t = t1-t0

evaluate('Overall Mean', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: -0.1953040317902126, rmse: 2.1979391673298037


#### Durchschnitt des Indikators für das jeweilige Jahr

In [20]:
def impute_yearly_means(df):
    #fill NaNs with overall mean of that indicator
    
    for i in df.columns:
        df[i] = df[i].fillna(df.groupby('Indicator Name')[i].transform('mean'))
            
    return df

In [21]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = impute_yearly_means(train)
t1 = time.time()

t = t1-t0

evaluate('Yearly Mean',df, t)

Mit dieser Methode bleiben 52772 NaNs bestehen.

17027 Werte wurden für die Metriken verwendet.
r2: 0.013454075014286193, rmse: 1.9969783912024721


#### Regionaler Durchschnitt des Indikators für das jeweilige Jahr

In [22]:
def impute_yearly_means_per_region(df):
    country_data = pd.read_csv('../Data/WDICountry.csv')
    country_data = country_data.loc[:,['Table Name', 'Region']]
    df = pd.merge(df.reset_index(), country_data, how='left', left_on='Country Name', right_on='Table Name').drop('Table Name', axis=1)
    df = df.set_index(['Country Name', 'Indicator Name', 'Region'])

    for i in df.columns:
        df[i] = df[i].fillna(df.groupby(['Indicator Name', 'Region'])[i].transform('mean'))

    df = df.reset_index().set_index(['Country Name', 'Indicator Name']).drop('Region', axis=1)
    return df

In [23]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = impute_yearly_means_per_region(train)
t1 = time.time()

t = t1-t0

evaluate('Yearly Mean per Region', df, t)

Mit dieser Methode bleiben 57448 NaNs bestehen.

16903 Werte wurden für die Metriken verwendet.
r2: 0.05990583802688387, rmse: 1.9565294158245126


#### Interpolation

In [24]:
def interpolate3(df):
    df = df.interpolate(limit=3)
    return df

def interpolate_all(df):
    df = df.interpolate()
    return df

In [26]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = interpolate3(train)
t1 = time.time()

t = t1-t0

evaluate('Interpolation 3', df, t)

Mit dieser Methode bleiben 33638 NaNs bestehen.

16488 Werte wurden für die Metriken verwendet.
r2: -0.04342564936248783, rmse: 2.0869887856001283


In [27]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = interpolate_all(train)
t1 = time.time()

t = t1-t0

evaluate('Interpolation all', df, t)

Mit dieser Methode bleiben 61 NaNs bestehen.

17027 Werte wurden für die Metriken verwendet.
r2: -0.04341923745526266, rmse: 2.053733699351703


#### Iterative Imputer

In [28]:
def iterative_imputer_1(df):
    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)
    return df

def iterative_imputer_2(df):
    df = df.unstack().T
    col = df.columns
    idx = df.index

    iter_imp = IterativeImputer(random_state=999)
    df= iter_imp.fit_transform(df)

    df = pd.DataFrame(df, columns=col, index=idx)
    df = df.unstack().T
    df = df.sort_index(level=['Country Name', 'Indicator Name'])
    
    return df

In [29]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = iterative_imputer_1(train)
t1 = time.time()

t = t1-t0

evaluate('Iterative Imputer 1', df, t)

C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: 0.9658526097359531, rmse: 0.37149693504215014


In [31]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = iterative_imputer_2(train)
t1 = time.time()

t = t1-t0

evaluate('Iterative Imputer 2', df, t)

Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: 0.9452737170229422, rmse: 0.4702994294809716


#### MICE

In [32]:
def mice_imputer(df):
    n_imputations =  12
    dfs = []
    
    for i in range(n_imputations): 
        print(f'Imputation round {i}')
        iter_imp = IterativeImputer(random_state=i, sample_posterior=True, verbose=2)
        df_temp = iter_imp.fit_transform(df)
        dfs.append(df_temp)
    
    df = np.mean(np.array(dfs), axis=0)
    return df

In [34]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = mice_imputer(train)
t1 = time.time()

t = t1-t0

Imputation round 0
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 35.73
[IterativeImputer] Ending imputation round 2/10, elapsed time 71.62
[IterativeImputer] Ending imputation round 3/10, elapsed time 107.78
[IterativeImputer] Ending imputation round 4/10, elapsed time 143.81
[IterativeImputer] Ending imputation round 5/10, elapsed time 179.54
[IterativeImputer] Ending imputation round 6/10, elapsed time 215.79
[IterativeImputer] Ending imputation round 7/10, elapsed time 251.72
[IterativeImputer] Ending imputation round 8/10, elapsed time 287.58
[IterativeImputer] Ending imputation round 9/10, elapsed time 323.50
[IterativeImputer] Ending imputation round 10/10, elapsed time 359.42
Imputation round 1
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 35.97
[IterativeImputer] Ending imputation round 2/10, elapsed time 71.98
[IterativeImputer

[IterativeImputer] Ending imputation round 8/10, elapsed time 303.26
[IterativeImputer] Ending imputation round 9/10, elapsed time 339.58
[IterativeImputer] Ending imputation round 10/10, elapsed time 388.68
Imputation round 11
[IterativeImputer] Completing matrix with shape (26070, 31)
[IterativeImputer] Ending imputation round 1/10, elapsed time 40.12
[IterativeImputer] Ending imputation round 2/10, elapsed time 78.45
[IterativeImputer] Ending imputation round 3/10, elapsed time 116.14
[IterativeImputer] Ending imputation round 4/10, elapsed time 152.37
[IterativeImputer] Ending imputation round 5/10, elapsed time 197.12
[IterativeImputer] Ending imputation round 6/10, elapsed time 243.68
[IterativeImputer] Ending imputation round 7/10, elapsed time 280.72
[IterativeImputer] Ending imputation round 8/10, elapsed time 319.67
[IterativeImputer] Ending imputation round 9/10, elapsed time 356.53
[IterativeImputer] Ending imputation round 10/10, elapsed time 393.15


In [35]:
evaluate('MICE', df, t)

C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: 0.9921502623704216, rmse: 0.17811645650204147


#### KNN Imputer

In [36]:
def knn_imputer(df):
    knn_imp = KNNImputer(n_neighbors=2)
    df= knn_imp.fit_transform(df)
    return df

In [37]:
base = reset_base()
train = reset_train()

t0 = time.time()
df = knn_imputer(train)
t1 = time.time()

t = t1-t0

evaluate('KNN Imputer', df, t)

C:\Users\mbfbr\Anaconda3\envs\other\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Mit dieser Methode bleiben 0 NaNs bestehen.

17030 Werte wurden für die Metriken verwendet.
r2: 0.40171197982598716, rmse: 1.555004425815101


#### Results

In [39]:
results = pd.DataFrame(results, columns=['Methode', 'r2', 'RSME', 'Remaining NaNs', 'Time'])

In [46]:
results = results.set_index('Methode')

In [47]:
results

,r2,RSME,Remaining NaNs,Time
Methode,,,,
Backfill,-0.043801,2.089040,33605,0.012711
Overall Mean,-0.195304,2.197939,0,0.697994
Yearly Mean,0.013454,1.996978,52772,0.199061
Yearly Mean per Region,0.059906,1.956529,57448,0.484859
Interpolation 3,-0.043426,2.086989,33638,0.197992
Interpolation all,-0.043419,2.053734,61,0.121871
Iterative Imputer 1,0.965853,0.371497,0,22.690486
Iterative Imputer 2,0.945274,0.470299,0,44.010055
MICE,0.992150,0.178116,0,4526.785384
